# COMP3221 - Week 10 - Replication

The goal of this lab is to integrate a multi-threaded TCP server with blockchain and test network connectivity.

## **Exercise 1: Blockchain and TCP server integration**


In the previous labs and assignments, you worked with TCP servers for communication. Integrate the Blockchain class with the TCP server. Make sure that there are no data races when storing the transactions from the clients that connect at the same time by using a synchronization primitive such as `threading.Lock`. Use the prefixed send and receive primitives provided in the Assignment 3 archive `network.py` file.

Use the following message format.

Sample transaction request.

In [ ]:
{
    "sender": "a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b",
    "message": "hello",
    "nonce": 0,
    "signature": "142e395895e0bf4e4a3a7c3aabf2f59d80c517d24bb2d98a1a24384bc7cb29c9d593ce3063c5dd4f12ae9393f3345174485c052d0f5e87c082f286fd60c7fd0c"
}

Sample transaction response.

In [ ]:
{
  "response": True
}

### Blockchain

In [ ]:
from cryptography.exceptions import InvalidSignature
import cryptography.hazmat.primitives.asymmetric.ed25519 as ed25519
from enum import Enum
import hashlib
import json
import re

sender_valid = re.compile('^[a-fA-F0-9]{64}$')
signature_valid = re.compile('^[a-fA-F0-9]{128}$')

TransactionValidationError = Enum('TransactionValidationError', ['INVALID_JSON', 'INVALID_SENDER', 'INVALID_MESSAGE', 'INVALID_SIGNATURE'])

def make_transaction(sender, message, signature) -> str:
	return json.dumps({'sender': sender, 'message': message, 'signature': signature})

def transaction_bytes(transaction: dict) -> bytes:
	return json.dumps({k: transaction.get(k) for k in ['sender', 'message']}, sort_keys=True).encode()

def make_signature(private_key: ed25519.Ed25519PrivateKey, message: str) -> str:
	transaction = {'sender': private_key.public_key().public_bytes_raw().hex(), 'message': message}
	return private_key.sign(transaction_bytes(transaction)).hex()

def validate_transaction(transaction: str) -> dict | TransactionValidationError:
	try:
		tx = json.loads(transaction)
	except json.JSONDecodeError:
		return TransactionValidationError.INVALID_JSON

	if not(tx.get('sender') and isinstance(tx['sender'], str) and sender_valid.search(tx['sender'])):
		return TransactionValidationError.INVALID_SENDER

	if not(tx.get('message') and isinstance(tx['message'], str) and len(tx['message']) <= 70 and tx['message'].isalnum()):
		return TransactionValidationError.INVALID_MESSAGE

	public_key = ed25519.Ed25519PublicKey.from_public_bytes(bytes.fromhex(tx['sender']))
	if not(tx.get('signature') and isinstance(tx['signature'], str) and signature_valid.search(tx['signature'])):
		return TransactionValidationError.INVALID_SIGNATURE
	try:
		public_key.verify(bytes.fromhex(tx['signature']), transaction_bytes(tx))
	except InvalidSignature:
		return TransactionValidationError.INVALID_SIGNATURE

	return tx


class Blockchain():
	def  __init__(self):
		self.blockchain = []
		self.pool = []
		self.new_block('0' * 64)

	def new_block(self, previous_hash=None):
		block = {
			'index': len(self.blockchain) + 1,
			'transactions': self.pool.copy(),
			'previous_hash': previous_hash or self.blockchain[-1]['current_hash'],
		}
		block['current_hash'] = self.calculate_hash(block)
		self.pool = []
		self.blockchain.append(block)

	def last_block(self):
		return self.blockchain[-1]

	def calculate_hash(self, block: dict) -> str:
		block_object: str = json.dumps({k: block.get(k) for k in ['index', 'transactions', 'previous_hash']}, sort_keys=True)
		block_string = block_object.encode()
		raw_hash = hashlib.sha256(block_string)
		hex_hash = raw_hash.hexdigest()
		return hex_hash

	def add_transaction(self, transaction: str) -> bool:
		if isinstance((tx := validate_transaction(transaction)), dict):
			self.pool.append(tx)
			return True
		return False


### Network

In [ ]:
import socket
import struct

def recv_exact(sock: socket.socket, msglen):
	chunks = []
	bytes_recd = 0
	while bytes_recd < msglen:
		chunk = sock.recv(min(msglen - bytes_recd, 2048))
		if chunk == b'':
			raise RuntimeError("socket connection broken")
		chunks.append(chunk)
		bytes_recd = bytes_recd + len(chunk)
	return b''.join(chunks)

def send_exact(sock: socket.socket, msg: bytes):
	totalsent = 0
	while totalsent < len(msg):
		sent = sock.send(msg[totalsent:])
		if sent == 0:
			raise RuntimeError("socket connection broken")
		totalsent = totalsent + sent

def recv_prefixed(sock: socket.socket):
	size_bytes = recv_exact(sock, 2)
	size = struct.unpack("!H", size_bytes)[0]
	if size == 0:
		raise RuntimeError("empty message")
	if size > 65535 - 2:
		raise RuntimeError("message too large")
	return recv_exact(sock, size)

def send_prefixed(sock: socket.socket, msg: bytes):
	size = len(msg)
	if size == 0:
		raise RuntimeError("empty message")
	if size > 65535 - 2:
		raise RuntimeError("message too large")
	size_bytes = struct.pack("!H", size)
	send_exact(sock, size_bytes + msg)


### **Client**

In [ ]:
import cryptography.hazmat.primitives.asymmetric.ed25519 as ed25519
import socket

from blockchain import make_signature, make_transaction
from network import recv_prefixed, send_prefixed

# TODO generate a private key and create a transaction using make_signature and make_transaction
# using the derived public key as the sender

# TODO create a socket and connect to the blockchain node

# TODO send the transaction, receive the response and print it


### **Server (Node)**

In [ ]:
from argparse import ArgumentParser
import json
from threading import Lock
import socketserver

from blockchain import Blockchain
from network import recv_prefixed, send_prefixed

class MyTCPServer(socketserver.ThreadingTCPServer):
	def __init__(self, server_address, RequestHandlerClass, bind_and_activate=True):
		# TODO initialize self.blockchain and self.blockchain_lock
		super().__init__(server_address, RequestHandlerClass, bind_and_activate)

class MyTCPHandler(socketserver.BaseRequestHandler):
	server: MyTCPServer

	def handle(self):
		# TODO create an infinite loop with the following steps
		# 1. receive data from the client using recv_prefixed and self.request
		# break to exit the loop in case of exception
		# 2. print the received data
		# 3. add the transaction to blockchain. use self.blockchain_lock to avoid data race
		# 4. send the response to the client using send_prefixed
		pass

if __name__ == '__main__':
	# TODO setup ArgumentParser with a required port argument of type int

	HOST = 'localhost'

	# TODO create an instance of MyTCPServer with the host, port, and MyTCPHandler
	# and use serve_forever to serve the requests. This will keep running until you
	# interrupt the program with Ctrl-C


#### Race Condition

 Two or more threads can access shared data and they try to change it at the same time.

<img src="https://i0.wp.com/contribute.geeksforgeeks.org/wp-content/uploads/race_condition.png" width=600>

→ The values of variables may be unpredictable and vary depending on the timings of context switches of the processes.

In [10]:
import threading
import time

# Shared data
counter = 0

def increment():
    global counter
    for _ in range(20000):
        # Adding a small sleep to force the CPU to switch between threads more often
        temp = counter
        time.sleep(0.00001)  # Introducing a tiny delay
        counter = temp + 1

# Creating threads
thread1 = threading.Thread(target=increment)
thread2 = threading.Thread(target=increment)

# Starting threads
thread1.start()
thread2.start()

# Waiting for both threads to finish
thread1.join()
thread2.join()

print(f"Expected counter value: 40000")
print(f"Actual counter value: {counter}")


Expected counter value: 40000
Actual counter value: 20001


#### Threading Lock

<img src="https://i0.wp.com/contribute.geeksforgeeks.org/wp-content/uploads/thread_sync.png" width=600>

In [11]:
import threading

# Shared data
counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(20000):
        with lock:
            # Ensuring only one thread can execute this block at once
            counter += 1

# Creating threads
thread1 = threading.Thread(target=increment)
thread2 = threading.Thread(target=increment)

# Starting threads
thread1.start()
thread2.start()

# Waiting for both threads to finish
thread1.join()
thread2.join()

print(f"Expected counter value: 40000")
print(f"Actual counter value: {counter}")


Expected counter value: 40000
Actual counter value: 40000


## **Exercise 2: Testing**

Try out the network communication with another student.

1. Make sure that you are connected to the same network, e.g. SSID UniSydney. Connect to [the university VPN](https://sydneyuni.service-now.com/sas?id=kb_article&sys_id=836b51e6dbeb6010ea7d0793f3961901) if you are not in the classroom. Determine the local IPv4 address of your machine, the first octet should read 10.

2. Create a client socket and connect it to the determined IP address of the other student.

3. Send a transaction and parse the response from the server.